## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-15-20-54-39 +0000,bbc,Jenrick defects to Reform UK after being sacke...,https://www.bbc.com/news/articles/cwyrpdy42n1o...
1,2026-01-15-20-51-00 +0000,wsj,Machado Leaves White House Visit With Little t...,https://www.wsj.com/world/americas/machado-lea...
2,2026-01-15-20-51-00 +0000,wsj,Trump Threatens to Invoke Insurrection Act in ...,https://www.wsj.com/politics/policy/trump-minn...
3,2026-01-15-20-50-33 +0000,nypost,Iran security forces raided hospital for wound...,https://nypost.com/2026/01/15/world-news/iran-...
4,2026-01-15-20-50-12 +0000,bbc,Ofsted inspects school over MP visit postponement,https://www.bbc.com/news/articles/cgrd9gyny81o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,69
22,iran,30
62,ice,21
292,new,18
248,greenland,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2026-01-15-20-04-06 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...,151
310,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...,146
29,2026-01-15-20-12-27 +0000,wapo,Trump threatens Insurrection Act deployment to...,https://www.washingtonpost.com/nation/2026/01/...,145
333,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...,144
189,2026-01-15-13-36-16 +0000,nypost,Trump threatens to invoke Insurrection Act if ...,https://nypost.com/2026/01/15/us-news/trump-th...,140


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,151,2026-01-15-20-04-06 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...
310,77,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...
333,70,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...
55,68,2026-01-15-19-20-00 +0000,wsj,"In an interview, the Philly Fed’s new presiden...",https://www.wsj.com/economy/central-banking/an...
3,55,2026-01-15-20-50-33 +0000,nypost,Iran security forces raided hospital for wound...,https://nypost.com/2026/01/15/world-news/iran-...
135,42,2026-01-15-16-34-29 +0000,nypost,Judge rejects Paramount Skydance request to sp...,https://nypost.com/2026/01/15/media/judge-reje...
125,41,2026-01-15-17-00-27 +0000,nypost,"Verizon offers ‘pathetic’ $20 credit, sparking...",https://nypost.com/2026/01/15/business/verizon...
159,41,2026-01-15-15-06-43 +0000,nypost,2 dead in Thailand after crane crashes onto ro...,https://nypost.com/2026/01/15/world-news/two-d...
5,37,2026-01-15-20-49-24 +0000,nypost,"Nick Reiner ordered into strict, involuntary m...",https://nypost.com/2026/01/15/us-news/nick-rei...
292,37,2026-01-15-02-13-51 +0000,cbc,Video shows woman dragged from car by ICE agen...,https://www.cbc.ca/news/world/ice-woman-car-mi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
